# Import basic packages (e.g., numpy and pandas)

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import datetime

# Read in and prepare the data set 

In [3]:
# read in the check-in file
data = pd.read_csv("/Users/KVTran/Documents/Research/WPI/Slip_Buddy/REU_2017/data/questions.csv")
# show the columns' names
list(data)
# select only a number of informative columns
col = ['DAY', 'patientid', 'date', 'Gender', 'Age', 'Marital Status', 'Education (Numeric)', 
       'Hispanic', 'Caucasian', 'Black', 'Asian', 'NHPI', 'Native American', 'Multi', 'Others',
       'Household', 'Income', 
       'AM Stress', 'AM Hunger', 'AM Sleep', 'AM Sleep hours',
       'AM Weight', 'Percent Weight change (from prev week)', 'Percent Weight change (from prev day)',
       'PM Stress','EVE Stress',
       'Number of Episodes Previous Day',
       'Episode']
data = data[col]
data.head()

,DAY,patientid,date,Gender,Age,Marital Status,Education (Numeric),Hispanic,Caucasian,Black,...,AM Hunger,AM Sleep,AM Sleep hours,AM Weight,Percent Weight change (from prev week),Percent Weight change (from prev day),PM Stress,EVE Stress,Number of Episodes Previous Day,Episode
0,1,7,11/24/2015,Male,52,Married,2,No,1,0,...,0.0,7.0,10.0,230.0,NaN,NaN,3.0,2.0,0,Y
1,2,7,11/25/2015,Male,52,Married,2,No,1,0,...,4.0,3.0,5.0,228.0,NaN,-0.008772,5.0,1.0,1,Y
2,3,7,11/26/2015,Male,52,Married,2,No,1,0,...,5.0,8.0,9.0,228.0,NaN,0.000000,2.0,NaN,1,Y
3,4,7,11/27/2015,Male,52,Married,2,No,1,0,...,5.0,7.0,8.0,229.0,NaN,0.004367,5.0,9.0,0,N
4,5,7,11/28/2015,Male,52,Married,2,No,1,0,...,5.0,5.0,5.0,229.0,NaN,0.000000,2.0,1.0,0,N


# Convert data to numerical

In [4]:
# Convert the categorical variables into numerical one
# Use the fit_transform() method from sklean.preprocessing.LableEncoder()
# Gender and Marital Status are catoegorical, not boolean. Hence, we add 1
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def toNumeric(data, nominalVariables):
    """
    This function takes in a list of categorical variables' names (columns' names) 
    It convert those values into numeric, and assign it directly to data dataframe
    Return nothing
    """
    for i in nominalVariables:
        numericalVals = le.fit_transform(data[i])
        if (i == "Gender" or i == "Marital Status"):
            numericalVals += 1
        data[i] = numericalVals
nominalVariables = ["Gender", "Marital Status","Hispanic", "Episode"]
toNumeric(data, nominalVariables)

# We remove the very last row that was added in when the data being converted to numerical
#data = data.drop(data.index[-1])

# Find out which columns contain NaN

In [5]:
# Check for NaN values in every columns
def checkNaN(data, col):  
    """
    This function accesses column to column and checks if there is any NaN
    If yes, print out the column
    """
    nullCount = 0
    colWithNaN = []
    for i in col:
        if data[i].isnull().any():
            colWithNaN.append(i)
            nullCount += 1
    if nullCount == 0:
        print("no null values found")
    else:
        return colWithNaN    
colWithNaN = checkNaN(data, col)
colWithNaN

['AM Stress',
 'AM Hunger',
 'AM Sleep',
 'AM Sleep hours',
 'AM Weight',
 'Percent Weight change (from prev week)',
 'Percent Weight change (from prev day)',
 'PM Stress',
 'EVE Stress']

# Adjust the Episode variable based on number of episode the day before

In [6]:
# The conflict between date format of Excel and Python was troublesome
# Issue solved by setting custom Excel format for date
# Choose the column -> format cell -> date -> choose m/d/yyyy -> choose custom -> mm/dd/yyy
def isConsecDate(date1, date2):
    """
    This function check on date1 and date2. Both are str() type
    If they are consecutive, return True
    """
    d = datetime.datetime.strptime(date1, '%m/%d/%Y') + datetime.timedelta(days=1)
    d = d.strftime('%m/%d/%Y')
    if (date2 == d):
        return True
    else:
        return False
    
def adjustEpisode(data):
    """
    
    """
    numRow = data.shape[0]
    for i in range(1, numRow):
        currentDate = data.date[i]
        prevDate = data.date[i-1]
        if isConsecDate(prevDate, currentDate):
            numEpPrevDay = data["Number of Episodes Previous Day"][i]
            if numEpPrevDay == 0:
                data.Episode.iloc[i-1] = 0
            else:
                data.Episode.iloc[i-1] = 1

adjustEpisode(data)
                


/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# Replace NaN with appropriate values

In [7]:
def replaceNaN(data, col):
    """
    Form a data block of rows as each patientid and each of the columns that contain NaN
    Compute the mean value of that data block
    replace the NaN value with that mean
    Return nothing
    """
    colWithNaN = checkNaN(data, col)
    patientID = list(set(data.patientid))
    for i in patientID:
        for j in colWithNaN:
            dataBlock = data[data.patientid == i][j]
            meanVal = np.mean(dataBlock)
            # replaceVal is a where() object of numpy.
            # check where the datablock contains NaN, replace that spot with meanVal.
            replaceVal = np.where(dataBlock.isnull(),
                                 -1,
                                 dataBlock)
            # Notice: using .loc[,] to assign value to the df itself instead of to a copy
            data.loc[data["patientid"] == i, j] = replaceVal

replaceNaN(data, col)

# Export the final data set

In [8]:
data.to_csv("/Users/KVTran/Documents/Research/WPI/Slip_Buddy/REU_2017/data/questions_cleaned.csv", index = False)